# Loan Risk Analysis - EDA
Bu notebook'ta loan risk verisi üzerinde temel keşifsel veri analizi yapacağız.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../src')
from data_loader import load_data

plt.style.use('default')
sns.set_palette('husl')

## 1. Veri Yükleme

In [ ]:
df = load_data()
print(f"Dataset shape: {df.shape}")
df.head()

## 2. Temel Bilgiler

In [ ]:
print("Veri Tipleri:")
print(df.dtypes)
print("\nEksik Değerler:")
print(df.isnull().sum())
print("\nTemel İstatistikler:")
df.describe()

## 3. Target Değişken Analizi

In [ ]:
# Target değişken dağılımı (loan verisinde genelde 'default', 'loan_status' gibi)
target_cols = [col for col in df.columns if any(word in col.lower() for word in ['default', 'status', 'target', 'class'])]
if target_cols:
    target_col = target_cols[0]
    print(f"Target column: {target_col}")
    print(df[target_col].value_counts())
    
    plt.figure(figsize=(8, 5))
    df[target_col].value_counts().plot(kind='bar')
    plt.title('Target Değişken Dağılımı')
    plt.show()
else:
    print("Target column bulunamadı. Tüm kolonları listeleyelim:")
    print(df.columns.tolist())

## 4. Sayısal Değişken Analizi

In [ ]:
# Sayısal kolonları bul
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Sayısal kolonlar: {numeric_cols}")

# Dağılımları görselleştir
if len(numeric_cols) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    for i, col in enumerate(numeric_cols[:6]):
        df[col].hist(bins=30, ax=axes[i])
        axes[i].set_title(f'{col} Dağılımı')
    
    plt.tight_layout()
    plt.show()

## 5. Kategorik Değişken Analizi

In [ ]:
# Kategorik kolonları bul
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"Kategorik kolonlar: {categorical_cols}")

# Her kategorik değişken için dağılım
for col in categorical_cols[:4]:  # İlk 4 kategorik değişken
    print(f"\n{col} dağılımı:")
    print(df[col].value_counts())
    
    plt.figure(figsize=(8, 4))
    df[col].value_counts().plot(kind='bar')
    plt.title(f'{col} Dağılımı')
    plt.xticks(rotation=45)
    plt.show()

## 6. Korelasyon Analizi

In [ ]:
# Sayısal değişkenler arası korelasyon
if len(numeric_cols) > 1:
    plt.figure(figsize=(10, 8))
    correlation_matrix = df[numeric_cols].corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title('Korelasyon Matrisi')
    plt.show()

## 7. Target vs Features Analizi

In [ ]:
# Target değişken ile diğer değişkenler arası ilişki
if 'target_col' in locals() and len(numeric_cols) > 1:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.ravel()
    
    numeric_features = [col for col in numeric_cols if col != target_col][:4]
    
    for i, col in enumerate(numeric_features):
        df.boxplot(column=col, by=target_col, ax=axes[i])
        axes[i].set_title(f'{col} by {target_col}')
    
    plt.tight_layout()
    plt.show()

## 8. Veri Kalitesi Kontrolü

In [ ]:
print("Veri Kalitesi Raporu:")
print(f"Toplam satır sayısı: {len(df)}")
print(f"Toplam kolon sayısı: {len(df.columns)}")
print(f"Eksik değer oranı: {(df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100:.2f}%")
print(f"Duplicate satır sayısı: {df.duplicated().sum()}")

# Aykırı değer kontrolü (basit IQR yöntemi)
print("\nAykırı Değer Analizi (IQR):")
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"{col}: {len(outliers)} aykırı değer ({len(outliers)/len(df)*100:.1f}%)")